In [7]:
import os
import sys
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.agents import create_agent
from pathlib import Path

load_dotenv()
api_key=os.environ.get("GOOGLE_API_KEY")

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [8]:
root_path = str(Path(os.getcwd()).resolve().parent) 
if root_path not in sys.path:
    sys.path.append(root_path)

In [9]:
# Create the agent
from tools.retrieve_and_reply import fast_search_engine
from langchain.agents import create_agent

tools = [fast_search_engine]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from the financial report. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [10]:
query = (
    "What is the R&D expense of GOOG in 2025 Q3"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the R&D expense of GOOG in 2025 Q3
================================== Ai Message ==================================
Tool Calls:
  fast_search_engine (c2591f54-e71e-45c1-af66-dd921d5999bd)
 Call ID: c2591f54-e71e-45c1-af66-dd921d5999bd
  Args:
    query: R&D expense of GOOG in 2025 Q3
================================= Tool Message =================================
Name: fast_search_engine

Research and development expenses for the three months ended September 30, 2025, were $15,151 million.
================================== Ai Message ==================================

[{'type': 'text', 'text': 'The R&D expense of GOOG in 2025 Q3 was $15,151 million.', 'extras': {'signature': 'CuMCAXLI2nwgdfyaE2UNGA8kaJHhxIfGpj57chCig5nvAVTCd0/R9FrkouVOut1yyJFYWl+MfudzUcf80Sq5rLkR6CiKan4Q/DlJjYVkPxWdl1lN2uIZRBX/QuPgYXaOA3gi/c50d/ELYMO9LI3oicXeoLE81UbKsPCPnOBBmY+PtUIOJd1xXPK5HDCWGpHdwRyz5gxt8kw3DD1CZlm5fb2jhzxloKA